In [ ]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import seaborn as sb
from scipy.stats import pareto
from scipy.stats import norm
from random import uniform
from sympy.abc import x
from sympy import Poly
from sympy.solvers.inequalities import solve_rational_inequalities
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib.collections import PolyCollection   # these are bunch of header files some aare needed some may be required for future use
import random
import pandas as pd

np.random.seed(0)
random.seed(0)

%matplotlib inline

In [ ]:
population_size = 100000
thetas = (1.15, 1.15, 1.8)
quantiles = np.linspace(0.00001,0.99999, population_size)

Z0 = pareto.ppf(quantiles, thetas[0])
Z1 = pareto.ppf(quantiles, thetas[1])
Z2 = pareto.ppf(quantiles, thetas[2])


fig, axs = plt.subplots(1,3, figsize = (25,5))

pd.Series(Z0).plot.kde(ax = axs[0], xlim = (0,1000))
pd.Series(Z1).plot.kde(ax = axs[1], xlim = (0,1000))
pd.Series(Z2).plot.kde(ax = axs[2], xlim = (0,1000))

axs[0].set_title('Distribution for Theta {}'.format(thetas[0]))
axs[1].set_title('Distribution for Theta {}'.format(thetas[1]))
axs[2].set_title('Distribution for Theta {}'.format(thetas[2]))

In [ ]:
W0, W1, W2 = Z0, Z1, Z2*5

A_high = 1
A_low = 0.9


Wage_high = A_high * W0
Wage_low = A_low * W0


discount_beta, return_R = 0.9, 1.05

In [ ]:
# BASED ON EQUATION XXX of PAPER
def calculate_savings(probability_wage_low, Wage_high, Wage_low):
    W_hat = probability_wage_low*Wage_high + (1-probability_wage_low)*Wage_low
    W_bar = Wage_high * Wage_low
    omega = Wage_low + Wage_high   + discount_beta *(W_hat - (W0* return_R))
    
    denominator = 2* (1+ discount_beta) * return_R
    numerator = -omega +np.sqrt((omega**2)-(4*(1+discount_beta)*W_bar) + (4 * discount_beta * (1+discount_beta) * return_R * W0 * W_hat ))
    return numerator/denominator


In [ ]:
# BASED ON EQUATION XXX of PAPER
def decision_function(probability_wage_low, savings , migration_cost, W0, Wage_low, Wage_high):
    sub_1 = (W0 - savings)
    sub_2 = (W0 - migration_cost)
    sub_3 = np.power((Wage_low + (return_R * migration_cost)), probability_wage_low)
    sub_4 = np.power((Wage_high + (return_R * migration_cost)), (1-probability_wage_low))
    
    output = np.multiply(sub_3, sub_4, np.power(np.divide(sub_1, sub_2), (1.0/discount_beta)))
    return np.power(output, (1.0/discount_beta))  

In [ ]:
probabilities_wage_low = [0.01,0.4, 0.8,0.8]
migration_costs = [(5,5,5,10),(10,10,10,20)]
A_lows = (0.2, 0.2, 0.2, 0.2)

fig,axs = plt.subplots(4,2 ,figsize=(25,20)) 
fig.suptitle("Fixed Migration Costs", fontsize=20, y=1.05)

for idx in range(8):
    r, c = idx//2, idx%2
    
    WL, WH, MC, p = A_lows[r]* W0, Wage_high, migration_costs[c][r], probabilities_wage_low[r]
    
    savings = calculate_savings(probability_wage_low = p, Wage_high = WH, Wage_low = WL)
    decision_function_out = decision_function(probability_wage_low = p, savings = savings, migration_cost = MC, W0 = W0, Wage_low = WL, Wage_high = WH)
    non_migration_wage = W1 if c ==0 else W2
    
    axs[r,c].plot(W0, decision_function_out, label = "Indifference Curve")
    axs[r,c].plot(W0, non_migration_wage, label = "Wage Distribution")

    txt = "P = {} \n MC = {} \n Al ={}".format(p, MC,  A_lows[r])
    axs[r,c].text(0.98,0.98, txt, bbox={'facecolor': 'aliceblue', 'alpha': 0.2, 'pad': 1},fontsize=15, transform=axs[r,c].transAxes, horizontalalignment='right', verticalalignment='top')
    
    axs[r,c].set_xlabel('Country {}'.format(c+1))
    axs[r,c].xaxis.set_label_position('top')
#     axs[r,c].set_xlim([0, 300])
#     axs[r,c].set_ylim([0, 300])
    axs[r,c].legend()
    plt.tight_layout()
